In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from fredapi import Fred



In [6]:
df = pd.read_csv("data.csv", sep=";", decimal=",")
df = df.rename(columns={
    "Column1": "Date",
    "Column2": "SPX",
    "Column3": "S5SFTW",
    "Column4": "S5PHRM",
    "Column5": "S5CPGS",
    "Column6": "S5ENRSX",
    "Column7": "S5FDBT",
    "Column8": "S5TECH",
    "Column9": "S5RETL",
    "Column10": "S5BANKX",
    "Column11": "S5HCES",
    "Column12": "S5DIVF",
    "Column13": "S5UTILX",
    "Column14": "S5MEDA",
    "Column15": "S5REAL",
    "Column16": "S5TELSX",
    "Column17": "S5MATRX",
    "Column18": "S5INSU",
    "Column19": "S5FDSR",
    "Column20": "S5HOUS",
    "Column21": "S5SSEQX",
    "Column22": "S5TRAN",
    "Column23": "S5HOTR",
    "Column24": "S5CODU",
    "Column25": "S5AUCO",
    "Column26": "S5COMS",
})
df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")



In [7]:
def GetReturn(df,date,lookback=180):
    date=pd.to_datetime(date)
    if date not in df["Date"].values:#add breaker if windows not in df
        raise ValueError("Date not in dataframe")
    returns_df = df[["Date","S5SFTW","S5PHRM","S5CPGS","S5ENRSX","S5FDBT","S5TECH","S5RETL","S5BANKX","S5HCES","S5DIVF","S5UTILX","S5MEDA","S5REAL","S5TELSX","S5MATRX","S5INSU","S5FDSR","S5HOUS","S5SSEQX","S5TRAN","S5HOTR","S5CODU","S5AUCO","S5COMS"]].copy()

    date_list=returns_df.drop(columns="Date")
    date_index = returns_df.index[returns_df["Date"] == date][0]
    returns_df=returns_df[(returns_df.index<=date_index) & (returns_df.index>=date_index-lookback) ]
    returns_df.drop(columns="Date",inplace=True)

    returns_df = np.log(returns_df/ returns_df.shift(1))
    returns_df.dropna(inplace=True)
    #print(returns_df.std().mean()) #verification if std is around 1% daily

    return returns_df


def GetReturnSPX(df,date,lookback=180):
    date=pd.to_datetime(date)
    if date not in df["Date"].values:#add breaker if windows not in df
        raise ValueError("Date not in dataframe")
    returns_df = df[["Date","SPX"]].copy()

    date_list=returns_df.drop(columns="Date")
    date_index = returns_df.index[returns_df["Date"] == date][0]
    returns_df=returns_df[(returns_df.index<=date_index) & (returns_df.index>=date_index-lookback) ]
    returns_df.drop(columns="Date",inplace=True)

    returns_df = np.log(returns_df/ returns_df.shift(1))
    returns_df.dropna(inplace=True)
    #print(returns_df.std().mean()) #verification if std is around 1% daily

    return returns_df

#Returns=GetReturn(df,"2020-05-11",lookback=180)
#ReturnsSPX=GetReturnSPX(df,"2020-05-11",lookback=180)

In [8]:
def GetSigma(df,date,lookback=180):
    returns_df=GetReturn(df,date,lookback=lookback)
    #covariance matric from returns_df
    sigma_windowed=returns_df.cov()

    return sigma_windowed

#Sigma=GetSigma(df,"2020-05-11",lookback=180)

In [9]:
def GetRfDataframe(df):
    fred = Fred(api_key="5c742a53d96bd3085e9199dcdb5af60b")
    riskfree = fred.get_series('DFF')
    # riskfree = fred.get_series('DTB1MO')

    riskfree = riskfree.to_frame(name='FedFunds')
    riskfree.index.name = "Date"
    riskfree = riskfree[riskfree.index >= "2002-01-01"]
    riskfree["FedFunds"]=riskfree["FedFunds"]/100
    list_days_open = pd.to_datetime(df["Date"], dayfirst=True, errors="coerce")
    list_days_full = pd.to_datetime(riskfree.index, dayfirst=True, errors="coerce")

    list_days_open=[pd.to_datetime(date) for date in list_days_open]
    list_days_full=[pd.to_datetime(date) for date in list_days_full]


    list_days_open_pondered=[]
    riskfree_list=[]
    count_list=[]
    timestamp=0
    while timestamp < len(list_days_full)-1:

      if list_days_full[timestamp+1] in list_days_open:
            list_days_open_pondered.append(list_days_full[timestamp])
            riskfree_list.append(riskfree["FedFunds"].loc[list_days_full[timestamp]])
            count_list.append(1)
            timestamp += 1

      else:
          count = 0
          timestampbis = timestamp
          while (timestamp + 1 < len(list_days_full)) and (list_days_full[timestamp + 1] not in list_days_open):
              timestamp += 1
              count += 1

          list_days_open_pondered.append(list_days_full[timestampbis])  # jour de départ
          riskfree_list.append(riskfree["FedFunds"].loc[list_days_full[timestampbis]])  # ✅ cohérent avec la date de départ
          count_list.append(count+1)
          timestamp += 1

    RfDf=pd.DataFrame({"Date":list_days_open_pondered,"Rf":riskfree_list,"Count":count_list})
    RfDf=RfDf.set_index("Date")
    return RfDf


def GetRiskFree(df,date,lookback=180):

    RfDf=GetRfDataframe(df)
    positionOfStartDate=df.index[df["Date"]==pd.to_datetime(date)][0]-lookback
    startDate=pd.to_datetime(df.iloc[positionOfStartDate,0])

    endDate=pd.to_datetime(date)
    RfDf=RfDf[(RfDf.index >= startDate) & (RfDf.index <= endDate )]
    CumulativeRf=[]

    for i in range(len(RfDf)):
      if i==0:
        CumulativeRf.append(pow((1+RfDf["Rf"].iloc[i]),(RfDf["Count"].iloc[i]/360)))
      else:
        CumulativeRf.append(pow((1+RfDf["Rf"].iloc[i]),(RfDf["Count"].iloc[i]/360))*CumulativeRf[i-1])

    RfDf["CumulativeRf"]=CumulativeRf
    RfDf["CumulativeRf"]= RfDf["CumulativeRf"]-1

    return RfDf["CumulativeRf"].iloc[-1]

#RiskFree=GetRiskFree(df,"2020-05-11",lookback=180)

In [10]:
def GetWeight(df,date):
    #for the moment we will use the equal weight
    weight_vector=np.zeros((24,1))
    for i in range(0,24):
        weight_vector[i]=1/24

    return weight_vector
#Weight=GetWeight(df,"2020-05-11")


In [18]:
def GetLambda(df,date,lookback=180):
    returns=GetReturn(df,date,lookback)
    returns=returns+1

    avg_return=returns.prod()-1 #geometric 180days return
    weight_vector=GetWeight(df=0,date=0)

    Sigma=GetSigma(df,date,lookback=180)

    var = float(weight_vector.T @ Sigma.values @ weight_vector)
    lambda_value=(avg_return@weight_vector - GetRiskFree(df,date,lookback=180))/np.sqrt(var)
    return lambda_value


#Lambda=GetLambda(df,"2016-05-11",lookback=180)


In [31]:
def GetPMatrix(df,date, lookback, proportion=3,offset=3):
    AssetColumns=["S5SFTW","S5PHRM","S5CPGS","S5ENRSX","S5FDBT","S5TECH","S5RETL","S5BANKX","S5HCES","S5DIVF","S5UTILX","S5MEDA","S5REAL","S5TELSX","S5MATRX","S5INSU","S5FDSR","S5HOUS","S5SSEQX","S5TRAN","S5HOTR","S5CODU","S5AUCO","S5COMS"]
    bestperformer = []
    worstperformer = []
    performerc = []
    returnBestPerformer=[]
    returnWorstPerformer=[]
    endDateIndex=df.index[df["Date"]==pd.to_datetime(date)][0]
    startDateIndex=df.index[df["Date"]==pd.to_datetime(date)][0]-lookback

    for i in range(1, df.shape[1]):  #loop through asset columns
        performerc.append((((float(df.iloc[endDateIndex, i]) / float(df.iloc[startDateIndex, i]) - 1) * 100), i - 1))

    performerc.sort(reverse=True)
    for i in range(proportion):
        bestperformer.append(performerc[i][1])
        returnBestPerformer.append(performerc[i][0])

    for i in range(len(performerc) - offset - proportion, len(performerc) - offset):
        worstperformer.append(performerc[i][1])
        returnWorstPerformer.append(performerc[i][0])

    P=np.zeros((1,24))
    for i in range(len(AssetColumns)):
        if i in bestperformer:
            P[0,i]=1/proportion
        elif i in worstperformer:
            P[0,i]=-1/proportion
        else:
            P[0,i]=0


    spreadLoosersWinnners=np.mean(returnBestPerformer)-np.mean(returnWorstPerformer)
    Q=np.array([[spreadLoosersWinnners/100]]) #convert to decimal
    return P, Q

#PMatrix,TempoQ=PMatrix(df,"2016-05-11",lookback=180)

In [20]:
def GetOmega(PMatrix, Sigma, tau=0.01):
    #Omega is the uncertainty of the views
    #we will use the simplest form : diagonal matrix with variance of the views
    #Omega = diag(P * (tau * Sigma) * P.T)

    tauSigma = tau * Sigma
    PMatrix_np = np.array(PMatrix)
    Omega_matrix = PMatrix_np @ tauSigma.values @ PMatrix_np.T

    #keep only diagonal elements
    Omega_diag = np.diag(np.diag(Omega_matrix))

    return Omega_diag

In [48]:
from scipy.optimize import minimize

def optimize_portfolio_markowitz(mu, Sigma, risk_aversion=1):
    """
    Optimisation de Markowitz avec contraintes réalistes.

    Parameters:
    -----------
    mu : array (N,) ou (N, 1)
        Rendements attendus (en décimal)
    Sigma : array (N, N)
        Matrice de covariance
    risk_aversion : float
        Paramètre d'aversion au risque

    Returns:
    --------
    weights : array (N, 1)
        Poids optimaux
    """
    n_assets = len(mu)

    # S'assurer que mu est un vecteur 1D
    if mu.ndim > 1:
        mu = mu

    # Fonction objectif : minimiser -utilité = -[mu^T w - (lambda/2) w^T Sigma w]
    def objective(w):
        portfolio_return = np.dot(mu, w)
        portfolio_variance = np.dot(w, np.dot(Sigma, w))
        utility = portfolio_return - (risk_aversion / 2) * portfolio_variance
        return -utility  # Minimize negative utility = maximize utility

    # Contraintes
    constraints = [
        # Les poids doivent sommer à 1 (fully invested)
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    ]

    # Bounds : limite les poids individuels
    bounds = [
        # Option 1: Long-only (pas de short)
        (0, 1) for _ in range(n_assets)

        # Option 2: Long/short limité (ex: entre -30% et +30%)
        # (-0.3, 0.3) for _ in range(n_assets)

        # Option 3: Long/short modéré
        # (-0.5, 0.5) for _ in range(n_assets)
    ]

    # Point de départ : equal weight
    w0 = np.ones(n_assets) / n_assets

    # Optimisation
    result = minimize(
        objective,
        w0,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints,
        options={'maxiter': 1000, 'ftol': 1e-9}
    )

    if not result.success:
        print(f"⚠️ Warning: Optimization did not converge: {result.message}")

    weights = result.x.reshape(-1, 1)

    # Vérifications
    print(f"\n{'='*60}")
    print(f"MARKOWITZ OPTIMIZATION RESULTS")
    print(f"{'='*60}")
    print(f"Success: {result.success}")
    print(f"Weights sum: {weights.sum():.6f} (should be 1.0)")
    print(f"Min weight: {weights.min():.4f}")
    print(f"Max weight: {weights.max():.4f}")
    print(f"Number of long positions: {(weights > 0.01).sum()}")
    print(f"Number of short positions: {(weights < -0.01).sum()}")

    # Top 5 positions
    sorted_idx = np.argsort(weights.flatten())[::-1]
    print(f"\nTop 5 positions:")
    for i in sorted_idx[:5]:
        print(f"  Asset {i}: {weights[i, 0]:.4f} ({weights[i, 0]*100:.2f}%)")

    print(f"{'='*60}\n")

    return weights

In [49]:
def BlackAndLittermanModel(backtestStartDate, rebalancingFrequency, lookbackPeriod, df):
    #implement the full backtest of the black and litterman model

    #---------
    #PARAMETERS
    #---------

    free_asset=0 #proportion of risk free asset allocated in the benchmark
    taux=0.01


    Sigma=GetSigma(df,backtestStartDate,lookback=lookbackPeriod)
    Lambda=GetLambda(df,backtestStartDate,lookback=lookbackPeriod)
    PMatrix,Q= GetPMatrix(df,backtestStartDate, lookback=lookbackPeriod, proportion=3,offset=3)
    Omega=GetOmega(PMatrix, Sigma, tau=taux)
    rf=GetRiskFree(df,backtestStartDate,lookback=lookbackPeriod)
    weights = GetWeight(df, backtestStartDate)
    weights = np.array(weights).reshape(-1, 1)
    uimplied = Lambda * (Sigma @ weights) + rf





    optimizedReturn=(np.linalg.inv(np.linalg.inv(taux*Sigma)+np.transpose(PMatrix)@np.linalg.inv(Omega)@PMatrix)) @ (np.linalg.inv(taux*Sigma)@uimplied+np.transpose(PMatrix)@np.linalg.inv(Omega)@Q)

    print("BL Returns",optimizedReturn)

    #MarkowitzAllocation

    WeightBL=np.linalg.inv(Sigma)@(optimizedReturn-rf)/Lambda

    print("BL Weights",WeightBL)

    optimal_weights = optimize_portfolio_markowitz(
        mu=optimizedReturn,  # Returns attendus de BL
        Sigma=Sigma,           # Matrice de covariance
        risk_aversion=Lambda
    )

    print(f"Optimal weights:\n{optimal_weights.flatten()}")














    pass

BlackAndLittermanModel("2016-05-11", rebalancingFrequency=3, lookbackPeriod=180, df=df)





C:\Users\mbard\AppData\Local\Temp\ipykernel_23016\4279076909.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  var = float(weight_vector.T @ Sigma.values @ weight_vector)


BL Returns            0
0   0.071833
1   0.061720
2   0.044357
3   0.082136
4   0.007666
5   0.133947
6   0.048955
7   0.081977
8   0.042205
9   0.069756
10 -0.039345
11  0.078935
12  0.010779
13  0.018415
14  0.061439
15  0.048145
16  0.028391
17  0.012569
18  0.115368
19  0.050083
20  0.050743
21  0.034074
22  0.033981
23  0.025572
BL Weights              0
0     0.041667
1     0.041667
2     0.041667
3     0.041667
4     0.041667
5   142.698103
6     0.041667
7     0.041667
8     0.041667
9     0.041667
10 -142.614769
11  142.698103
12    0.041667
13    0.041667
14    0.041667
15    0.041667
16    0.041667
17    0.041667
18  142.698103
19    0.041667
20    0.041667
21    0.041667
22 -142.614769
23 -142.614769


ValueError: shapes (24,1) and (24,) not aligned: 1 (dim 1) != 24 (dim 0)